# Sequential Parameter Optimization
## `river` Hyperparameter Tuning: Nowcasting


# Example: Nowcasting Hyperparameter

In [ ]:
import numpy as np
import datetime as dt
import pandas as pd
import statistics
from math import inf
from scipy.optimize import shgo
from scipy.optimize import direct
from scipy.optimize import differential_evolution
from matplotlib import pyplot
import matplotlib.pyplot as plt
from river import anomaly
from river import compose
from river import preprocessing
from river import linear_model
from river import datasets
from river import metrics
from river import time_series
from river import utils
from spotRiver.fun.hyperriver import HyperRiver
from spotPython.spot import spot

## 1 Nowcasting Example from River (Airline Passengers)

* This is the Nowcasting example taken from [https://riverml.xyz/0.15.0/examples/building-a-simple-nowcasting-model/](https://riverml.xyz/0.15.0/examples/building-a-simple-nowcasting-model/)
* It is used to check whether the implementation can be executed.
* Finally, the objective function is added.

In [ ]:
from river import compose
from river import linear_model
from river import preprocessing, datasets, utils, metrics
import matplotlib.pyplot as plt
from spotRiver.utils.features import get_ordinal_date
from spotRiver.evaluation.eval_nowcast import eval_nowcast_model, plot_nowcast_model

dataset = datasets.AirlinePassengers()
for x, y in datasets.AirlinePassengers():
    print(x, y)
    break


In [ ]:


model = compose.Pipeline(
    ('ordinal_date', compose.FuncTransformer(get_ordinal_date)),
    ('scale', preprocessing.StandardScaler()),
    ('lin_reg', linear_model.LinearRegression())
)

dates, metric, y_trues, y_preds = eval_nowcast_model(model, dataset=dataset)
plot_nowcast_model(dates, metric, y_trues, y_preds)

## 2 Nowcasting: The Improved Model for Airline Passengers

In [ ]:
from river import compose
from river import linear_model
from river import preprocessing, datasets, utils, metrics
import matplotlib.pyplot as plt
from spotRiver.utils.features import get_month_distances
from spotRiver.evaluation.eval_nowcast import eval_nowcast_model, plot_nowcast_model
from river import optim

dataset = datasets.AirlinePassengers()

model = compose.Pipeline(
    ('features', compose.TransformerUnion(
        ('ordinal_date', compose.FuncTransformer(get_ordinal_date)),
        ('month_distances', compose.FuncTransformer(get_month_distances)),
    )),
    ('scale', preprocessing.StandardScaler()),
    ('lin_reg', linear_model.LinearRegression(
        intercept_lr=0,
        optimizer=optim.SGD(0.03)
    ))
)

dates, metric, y_trues, y_preds = eval_nowcast_model(model, dataset=dataset)

plot_nowcast_model(dates, metric, y_trues, y_preds)

## 3 Nowcasting: The Improved Model for GW Data

In [ ]:
import importlib.resources as pkg_resources
import spotRiver.data as data
inp_file = pkg_resources.files(data)
csv_path = str(inp_file.resolve())
csv_path

In [1]:
from spotRiver.data.generic import GenericData
import importlib.resources as pkg_resources
import spotRiver.data as data
inp_file = pkg_resources.files(data)
csv_path = str(inp_file.resolve())


dataset = GenericData(filename="UnivariateData.csv",
                      directory=csv_path,
                      target="Consumption",
                      n_features=1,
                      n_samples=51_706,
                      converters={"Consumption": float},
                      parse_dates={"Time": "%Y-%m-%d %H:%M:%S%z"})
for x, y in dataset:
    print(x, y)
    break

{'Time': datetime.datetime(2016, 12, 31, 23, 0, tzinfo=datetime.timezone.utc)} 10951.217


In [ ]:
from river import compose
from river import linear_model
from river import preprocessing, datasets, utils, metrics
import matplotlib.pyplot as plt
from spotRiver.utils.features import get_month_distances, get_weekday_distances, get_ordinal_date, get_hour_distances
from spotRiver.evaluation.eval_nowcast import eval_nowcast_model, plot_nowcast_model
from river import optim
from spotRiver.data.generic import GenericData
import importlib.resources as pkg_resources
import spotRiver.data as data
inp_file = pkg_resources.files(data)
csv_path = str(inp_file.resolve())


dataset = GenericData(filename="UnivariateData.csv",
                      directory=csv_path,
                      target="Consumption",
                      n_features=1,
                      n_samples=51_706,
                      converters={"Consumption": float},
                      parse_dates={"Time": "%Y-%m-%d %H:%M:%S%z"})
for x, y in dataset:
    print(x, y)
    break


In [ ]:
model = compose.Pipeline(
    ('features', compose.TransformerUnion(
        ('ordinal_date', compose.FuncTransformer(get_ordinal_date)),
        ('month_distances', compose.FuncTransformer(get_month_distances)),
        ('weekday_distances', compose.FuncTransformer(get_weekday_distances)),
        ('hour_distances', compose.FuncTransformer(get_hour_distances)),
    )),
    ('scale', preprocessing.StandardScaler()),
    ('lin_reg', linear_model.LinearRegression(
        intercept_lr=0,
        optimizer=optim.SGD(0.001)
    ))
)
# model = preprocessing.TargetStandardScaler(regressor=model)

In [ ]:
dates, metric, y_trues, y_preds = eval_nowcast_model(model, dataset=dataset, time_interval="Time")

In [ ]:
metric.get()

In [ ]:
plot_nowcast_model(dates, metric, y_trues, y_preds, range=[51_600, -1])

In [ ]:
model['lin_reg'].weights
# model.regressor['lin_reg'].weights

# 4 SPOT Hyperparameter

In [ ]:
import numpy as np
from math import inf
from river import compose
from river import linear_model
from river import preprocessing, datasets, utils, metrics
import matplotlib.pyplot as plt
from spotRiver.utils.features import get_month_distances, get_weekday_distances, get_ordinal_date, get_hour_distances
from spotRiver.evaluation.eval_nowcast import eval_nowcast_model, plot_nowcast_model
from river import optim
from scipy.optimize import differential_evolution
from spotRiver.data.generic import GenericData
from spotRiver.fun.hyperriver import HyperRiver
from spotPython.spot import spot


dataset = GenericData(filename="UnivariateData.csv",
                      directory="/Users/bartz/data/",
                      target="Consumption",
                      n_features=1,
                      n_samples=51_706,
                      converters={"Consumption": float},
                      parse_dates={"Time": "%Y-%m-%d %H:%M:%S%z"})

fun = HyperRiver(123).fun_nowcasting
var_name = ["lr", "intercept_lr", "hour", "weekday", "month"]
var_type = ["num", "num"] + ["factor"] * 3
lower = np.array([0.00225, 0.01,        0,      0,   0])
upper = np.array([0.003,  0.09,       1,      1,   1])
fun_control = {"data": dataset}

In [ ]:
spot_now = spot.Spot(fun=fun,
                   lower = lower,
                   upper = upper,
                   fun_evals = inf,
                   fun_repeats = 1,
                   max_time = 360,
                   noise = False,
                   tolerance_x = np.sqrt(np.spacing(1)),
                   var_type=var_type,
                   var_name=var_name,
                   infill_criterion = "y",
                   n_points = 1,
                   seed=123,
                   log_level = 50,
                   show_models= False,
                   show_progress= True,
                   fun_control = fun_control,
                   design_control={"init_size": 20,
                                   "repeats": 1},
                   surrogate_control={"noise": True,
                                      "cod_type": "norm",
                                      "min_theta": -4,
                                      "max_theta": 3,
                                      "n_theta": 5,
                                      "model_optimizer": differential_evolution,
                                      "model_fun_evals": 1000,
                                      "log_level": 50
                                      })
spot_now.run()

In [ ]:
spot_now.plot_progress(log_y=True)

In [ ]:
spot_now.print_results()

In [ ]:
spot_now.print_importance()

In [ ]:
X = spot_now.to_all_dim(spot_now.min_X.reshape(1,-1))
print(X)
lr = X[:, 0]
intercept_lr = X[:, 1]
hour = X[:, 2]
weekday = X[:, 3]
month = X[:, 4]

In [ ]:
h_i = int(hour)
w_i = int(weekday)
m_i = int(month)
# baseline:
extract_features = compose.TransformerUnion(get_ordinal_date)
if h_i:
    extract_features = compose.TransformerUnion(get_ordinal_date, get_hour_distances)
if w_i:
    extract_features = compose.TransformerUnion(extract_features, get_weekday_distances)
if m_i:
    extract_features = compose.TransformerUnion(extract_features, get_month_distances)
model_spot = compose.Pipeline(
    ("features", extract_features),
    ("scale", preprocessing.StandardScaler()),
    (
        "lin_reg",
        linear_model.LinearRegression(
            intercept_init=0, optimizer=optim.SGD(float(lr)), intercept_lr=float(intercept_lr)
        ),
    ),
)
# eval:
dates, metric, y_trues, y_preds = eval_nowcast_model(
    model_spot, dataset=dataset, time_interval="Time"
)
z = metric.get()
z

In [ ]:
plot_nowcast_model(dates, metric, y_trues, y_preds, range=[51_606, -1])

In [ ]:
model_spot['lin_reg'].weights

In [ ]:
min_z = None
max_z = None


* For productive use, you might want to select:
  * `min_z=min(spot_now.y)` and
  * `max_z = max(spot_now.y)`
* These settings are not so colorful, but give better insights.

In [ ]:
min(spot_now.y), max(spot_now.y)
n = spot_now.k
for i in range(n-1):
    for j in range(i+1, n):
        spot_now.plot_contour(i=i, j=j, min_z=min_z, max_z = max_z)